# Q&A Chatbot

In [2]:
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

llm = GooglePalm(temperature=0.8)

/Users/chihyi/Documents/CL_SS_24/QA-Chatbot/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# prompt the LLM
poem = llm("Write a 4 line poem of stinky tofu")
print(poem)

/Users/chihyi/Documents/CL_SS_24/QA-Chatbot/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


**Stinky Tofu**

Smelly and delicious,
Stinky tofu is a delicacy.
A must-try for adventurous eaters,
It's an acquired taste.


### Load data

In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="codebasics_faqs.csv", source_column="prompt", encoding='cp1252')
data = loader.load()

### Embedding model - Hugging face instructor embeddings

In [8]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-large')

load INSTRUCTOR_Transformer
max_seq_length  512


### Vector store - FAISS

In [9]:
from langchain.vectorstores import FAISS

# create a vector database from data
vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)

In [14]:
query = "How long can I access the course?"
# create a retriever for querying the vector database
retriever = vectordb.as_retriever()
docs = retriever.invoke(query)

In [15]:
print(docs[0].page_content)

prompt: Once purchased, is this course available for lifetime access?
response: Yes


In [17]:
docs = retriever.get_relevant_documents("any support for finding job?")
print(docs[0].page_content)

prompt: Do you provide any job assistance?
response: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.


### **Create RetrievalQA chain along with prompt template**

In [23]:
from langchain_core.prompts import ChatPromptTemplate

template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {input}"""

prompt = ChatPromptTemplate.from_template(template)

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [25]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x106ec6550>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Given the following context and a question, generate an answer based on this context only.\nIn the answer try to provide as much text as possible from "response" section in the source document context without making much changes.\nIf the answer is not found in the context, kindly st

In [27]:
response = retrieval_chain.invoke({"input": "Do you offer javascript course?"})
print(response)

{'input': 'Do you offer javascript course?', 'context': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}), Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}), Document(page_content='prompt: What is 

In [28]:
response['answer']

"I don't know."